In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openai SQLAlchemy

In [ ]:
from openai import OpenAI
OPENAI_API_KEY = userdata.get('OPENAI')

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import json
from openai import OpenAI
import os
from sqlalchemy import (
    create_engine, Column, Integer, String, Text
)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import re

def extract_json_from_response(text: str) -> dict:
    """
    Given a ChatGPT response that contains a ```json ... ``` block,
    finds the first JSON object inside and returns it as a Python dict.
    """
    # This will match ```json { ... } ```
    pattern = r"```json\s*(\{.*?\})\s*```"
    match = re.search(pattern, text, re.DOTALL)
    if not match:
        return json.loads(text)
    json_str = match.group(1)
    return json.loads(json_str)

# ── 1. Configure OpenAI ───────────────────────────────────────────────────────
def fetch_ai_data(term: str,client) -> dict:
    prompt = f"""
You are an excellent English Speaker. Given the input term: «{term}», do the following:
1) Detect if it's a single word or a multi-word phrase.
2) If it's a phrase, classify it as either:
   - include a phrasal verb (then extract the verb part into "verb"), or
   - an idiom (set "verb" to null).
3) Produce:
   • "meaning": a one-sentence definition.
   • "examples": one example sentences.
   • "alternatives": (if phrase) an array of two alternative phrasings. (if word) an array of two synonyms.

Return JSON file exactly in this content:
  "type": "word" | "phrase",
  "verb": string|null,
  "meaning": string,
  "examples": string,
  "alternatives": string, string
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return extract_json_from_response(response.choices[0].message.content)

# ── 2. Define our SQL schema via SQLAlchemy ──────────────────────────────────
Base = declarative_base()

class Word(Base):
    __tablename__ = 'word'
    id      = Column(Integer, primary_key=True, autoincrement=True)
    word    = Column(String, nullable=False, unique=True)
    meaning = Column(Text,   nullable=False)
    examples    = Column(Text,   nullable=False)
    alternatives = Column(Text,   nullable=False)  # we'll JSON‐dump

class Phrase(Base):
    __tablename__ = 'phrase'
    id          = Column(Integer, primary_key=True, autoincrement=True)
    phrase      = Column(String, nullable=False) # unique=False because we want to allow same phrases with different meanings
    verb        = Column(String, nullable=True)
    meaning     = Column(Text,   nullable=False)
    examples    = Column(Text,   nullable=False)
    alternatives = Column(Text,   nullable=False)  # JSON‐dump

db_path = '/content/drive/MyDrive/Colab Notebooks/flashcards.db'
engine = create_engine(f'sqlite:///{db_path}')
Session = sessionmaker(bind=engine)
if not os.path.exists(db_path):
    Base.metadata.create_all(engine)


<ipython-input-45-b7a8e2d4c231>:56: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
import sqlite3

con = sqlite3.connect(db_path)
cur = con.cursor()
res = cur.execute("PRAGMA table_info(phrase);")
print(res.fetchall())
con.close()

[(0, 'id', 'INTEGER', 1, None, 1), (1, 'phrase', 'VARCHAR', 1, None, 0), (2, 'verb', 'VARCHAR', 0, None, 0), (3, 'meaning', 'TEXT', 1, None, 0), (4, 'examples', 'TEXT', 1, None, 0), (5, 'alternatives', 'TEXT', 1, None, 0)]


In [ ]:
import pandas as pd
# ── 3. Read CSV → AI → DB ────────────────────────────────────────────────────
def process_csv(path):
    session = Session()
    client = OpenAI(api_key=OPENAI_API_KEY)

    worksheet = gc.open(path).sheet1

    # get_all_values gives a list of rows.
    rows = worksheet.get_all_values()
    for row in rows:
        term = row[0].strip()
        if not term:
            continue

        if len(row) == 2 and row[1] == '1':
            print(f"Term '{term}' already stored, skipping.")
            continue

        data = fetch_ai_data(term,client)
        if data["type"] == "word":
            w = Word(
                word    = term,
                meaning = data["meaning"],
                examples = data["examples"],
                alternatives = json.dumps(data["alternatives"], ensure_ascii=False)
            )
            session.merge(w)
            print(f"Term '{term}' Added to Word")

        else:
            p = Phrase(
                phrase      = term,
                verb        = data["verb"],  # or None
                meaning     = data["meaning"],
                examples    = data["examples"],
                alternatives = json.dumps(data["alternatives"], ensure_ascii=False)
            )
            session.merge(p)
            print(f"Term '{term}' Added to Phrase")
        row[1] = '1'
    session.commit()

    cell_list = worksheet.range(f'B1:B{len(rows)}')

    for idx, cell in enumerate(cell_list):
      cell.value = rows[idx][1]

    worksheet.update_cells(cell_list)

    session.close()
    print("All terms processed and stored into SQL.")


process_csv('FlashCardApp')


Term 'Knock them dead' Added to Phrase
Term 'Evidently' Added to Word
Term 'That explains it' Added to Phrase
Term 'Insist' Added to Word
Term 'In the first place' Added to Phrase
Term 'As evidenced by my experiecence of' Added to Phrase
Term 'Go down the tube' Added to Phrase
Term 'as a matter of fact' Added to Phrase
Term 'The ball is in your court' Added to Phrase
Term 'knock yourself out' Added to Phrase
Term 'That's not the point' Added to Phrase
Term 'Look so blue' Added to Phrase
Term 'That says it all' Added to Phrase
Term 'Put it there' Added to Phrase
Term 'Swamped' Added to Word
Term 'Back off' Added to Phrase
Term 'Put my ass on the line' Added to Phrase
Term 'Pity' Added to Word
Term 'Self assured' Added to Phrase
Term 'keep your eye on' Added to Phrase
Term 'Have the ball' Added to Phrase
Term 'Just so you know' Added to Phrase
Term 'Set my sight on' Added to Phrase
Term 'Kick someone's ass' Added to Phrase
Term 'Push on me' Added to Phrase
Term 'As I recall / Which I rec